In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Feedback or issues?
For any feedback or questions, please open an [issue](https://github.com/googleapis/python-aiplatform/issues).

# Vertex SDK for Python: Custom Training and AutoML Tabular Training Example

To use this Colaboratory notebook, you copy the notebook to your own Google Drive and open it with Colaboratory (or Colab). You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Colab automatically displays the return value of the last line in each cell. For more information about running notebooks in Colab, see the [Colab welcome page](https://colab.research.google.com/notebooks/welcome.ipynb).

This notebook demonstrate how to create both an AutoML model and a custom model based on a tabular dataset. It will require you provide a bucket where the dataset will be stored.

Note: you may incur charges for training, prediction,  storage or usage of other GCP products in connection with testing this SDK.

# Install Vertex SDK for Python, Authenticate, and upload of a Dataset to your GCS bucket


After the SDK installation the kernel will be automatically restarted. You may see this error message `Your session crashed for an unknown reason` which is normal.

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import sys

if "google.colab" in sys.modules:
    import os

    from google.colab import auth

    auth.authenticate_user()
    os.environ["GOOGLE_CLOUD_PROJECT"] = MY_PROJECT

### Enter your project and GCS bucket

Enter your Project Id in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

In [ ]:
MY_PROJECT = "YOUR PROJECT"
MY_STAGING_BUCKET = "gs://YOUR BUCKET"  # bucket should be in same region as Vertex AI

The dataset we are using is the California Housing Dataset, available locally in Colab. For more information about this dataset and its license, please visit: https://developers.google.com/machine-learning/crash-course/california-housing-data-description

In [ ]:
!gcloud config set project {MY_PROJECT}

TRAIN_FILE_NAME = "california_housing_train.csv"
!gsutil cp sample_data/{TRAIN_FILE_NAME} {MY_STAGING_BUCKET}/data/

gcs_csv_path = f"{MY_STAGING_BUCKET}/data/{TRAIN_FILE_NAME}"

# Initialize Vertex SDK for Python

Initialize the *client* for Vertex AI

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

# Create Managed Tabular Dataset from CSV

In [ ]:
ds = aiplatform.TabularDataset.create(
    display_name="housing", gcs_source=[gcs_csv_path], sync=False
)

# Write your Training Script
- Write this cell as a file which will be used for custom training.

In [ ]:
%%writefile training_script.py

import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


# uncomment and bump up replica_count for distributed training
# strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
# tf.distribute.experimental_set_strategy(strategy)

target = 'median_house_value'

def aip_data_to_dataframe(wild_card_path):
    return pd.concat([pd.read_csv(fp.numpy().decode())
                      for fp in tf.data.Dataset.list_files([wild_card_path])])

def get_features_and_labels(df):
    features = df.drop(target, axis=1) 
    return {key: features[key].values for key in features.columns}, df[target].values

def data_prep(wild_card_path):
    return get_features_and_labels(aip_data_to_dataframe(wild_card_path))

train_features, train_labels = data_prep(os.environ["AIP_TRAINING_DATA_URI"])

feature_columns = [tf.feature_column.numeric_column(name) for name in 
                   train_features.keys()]

model = tf.keras.Sequential([
            layers.DenseFeatures(feature_columns),
            layers.Dense(64),
            layers.Dense(1)])
model.compile(loss='mse', optimizer='adam')

model.fit(train_features, train_labels,
          epochs=10,
          validation_data=data_prep(os.environ["AIP_VALIDATION_DATA_URI"]))
print(model.evaluate(*data_prep(os.environ["AIP_TEST_DATA_URI"])))

# save as Vertex AI Managed model
tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])

# Launch Training AutoML and Custom Training Jobs to Create Models

Once we have created your dataset and defined your training script, we will create a model.

In [ ]:
custom_job = aiplatform.CustomTrainingJob(
    display_name="train-housing",
    script_path="training_script.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest",
    requirements=[
        "gcsfs==0.7.1",
    ],
    model_serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest",
)

custom_model = custom_job.run(
    ds, replica_count=1, model_display_name="housing-model", sync=False
)

automl_job = aiplatform.AutoMLTabularTrainingJob(
    display_name="train-housing-automl_1",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=[
        {"numeric": {"column_name": "longitude"}},
        {"numeric": {"column_name": "latitude"}},
        {"numeric": {"column_name": "housing_median_age"}},
        {"numeric": {"column_name": "total_rooms"}},
        {"numeric": {"column_name": "total_bedrooms"}},
        {"numeric": {"column_name": "population"}},
        {"numeric": {"column_name": "households"}},
        {"numeric": {"column_name": "median_income"}},
    ],
    optimization_objective_recall_value=None,
    optimization_objective_precision_value=None,
)

# This will take around an hour to run
automl_model = automl_job.run(
    dataset=ds,
    target_column="median_house_value",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name="house-value-prediction-model",
    disable_early_stopping=False,
    predefined_split_column_name=None,
    sync=False,
)

# Deploy Model

In [ ]:
custom_endpoint = custom_model.deploy(machine_type="n1-standard-4", sync=False)
automl_endpoint = automl_model.deploy(machine_type="n1-standard-4", sync=False)

# Predict on Endpoint

In [ ]:
custom_endpoint.wait()

In [ ]:
# This sample is taken from an observation where median_house_value = 94600
prediction = custom_endpoint.predict(
    [
        {
            "longitude": -124.35,
            "latitude": 40.54,
            "housing_median_age": 52.0,
            "total_rooms": 1820.0,
            "total_bedrooms": 300.0,
            "population": 806,
            "households": 270.0,
            "median_income": 3.014700,
        },
    ]
)
prediction

In [ ]:
automl_job.state

In [ ]:
automl_endpoint.wait()

In [ ]:
# This sample is taken from an observation where median_house_value = 94600
prediction = automl_endpoint.predict(
    [
        {
            "longitude": "-124.35",
            "latitude": "40.54",
            "housing_median_age": "52.0",
            "total_rooms": "1820.0",
            "total_bedrooms": "300.0",
            "population": "806",
            "households": "270.0",
            "median_income": "3.014700",
        },
    ]
)
prediction